In [5]:
import random
import pandas as pd
from collections import defaultdict
from datetime import datetime, timedelta

def assign_dates(members, dates, previous_dates, assigned_members, min_gap_days=7):
    """
    Assigns dates to members ensuring that members from the same group do not appear consecutively
    and that their new assignment is not too close to their previous talk date.
    Also ensures no member is assigned more than once.
    
    :param members: List of tuples (name, group)
    :param dates: List of dates to assign
    :param previous_dates: Dictionary mapping member names to their previous talk date (YYYY-MM-DD format)
    :param assigned_members: Set of members already assigned to avoid duplicates
    :param min_gap_days: Minimum number of days between talks for the same member
    :return: Dictionary mapping dates to assigned members
    """
    previous_dates = {name: datetime.strptime(date, "%d/%m/%Y") for name, date in previous_dates.items() if isinstance(date, str) and pd.notna(date)}
    groups = defaultdict(list)
    for name, group in members:
        if isinstance(name, str) and name.strip() and name not in assigned_members:  # Avoid already assigned members and blank spaces
            groups[group].append(name)
    for group in groups:
        random.shuffle(groups[group])
    group_list = list(groups.items())
    random.shuffle(group_list)
    assignments = {}
    last_group = None
    for date in dates:
        date_obj = datetime.strptime(date, "%Y-%m-%d")
        for i, (group, members) in enumerate(group_list):
            for member in members:
                if (group != last_group and member not in assigned_members and
                        (member not in previous_dates or (date_obj - previous_dates[member]).days >= min_gap_days)):
                    assignments[date] = member
                    last_group = group
                    members.remove(member)
                    assigned_members.add(member)  # Track assigned members
                    previous_dates[member] = date_obj
                    break
            else:
                continue
            break
        else:
            random.shuffle(group_list)
            for group, members in group_list:
                for member in members:
                    if isinstance(member, str) and member.strip() and member not in assigned_members and (member not in previous_dates or (date_obj - previous_dates[member]).days >= min_gap_days):
                        assignments[date] = member
                        last_group = group
                        members.remove(member)
                        assigned_members.add(member)  # Track assigned members
                        previous_dates[member] = date_obj
                        break
                else:
                    continue
                break
    return assignments

def load_data(file_path):
    """Load members and previous dates from an Excel file."""
    xls = pd.ExcelFile(file_path)
    df_sheet1 = pd.read_excel(xls, sheet_name="Sheet1")
    df_sheet2 = pd.read_excel(xls, sheet_name="Sheet2")
    df_sheet2 = df_sheet2.rename(columns=lambda x: x.strip())
    members = list(zip(df_sheet1['Name'], df_sheet1['Allocation']))
    previous_dates = df_sheet1.set_index('Name')['Date of last seminar'].dropna().astype(str).to_dict()
    df_sheet2["Date"] = pd.to_datetime(df_sheet2["Date"], format="%d/%m/%Y")
    unassigned_dates = df_sheet2[df_sheet2["Person"].isna()]["Date"].dt.strftime("%Y-%m-%d").tolist()
    assigned_members = set(df_sheet2["Person"].dropna().astype(str).str.strip().tolist())  # Collect already assigned members, removing blanks
    return members, previous_dates, df_sheet1, df_sheet2, unassigned_dates, assigned_members

def save_assignments(df_sheet1, df_sheet2, output_file):
    """Save the assignment results to an Excel file."""
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        df_sheet1.to_excel(writer, sheet_name="Sheet1", index=False)
        df_sheet2.to_excel(writer, sheet_name="Sheet2", index=False)

input_file = "Book1.xlsx"
output_file = "updated_assignments.xlsx"
members, previous_dates, df_sheet1, df_sheet2, unassigned_dates, assigned_members = load_data(input_file)
assigned_new_members = assign_dates(members, unassigned_dates, previous_dates, assigned_members)
for i, row in df_sheet2.iterrows():
    date_str = row["Date"].strftime("%Y-%m-%d")
    if pd.isna(row["Person"]) and date_str in assigned_new_members:
        df_sheet2.at[i, "Person"] = assigned_new_members[date_str]
save_assignments(df_sheet1, df_sheet2, output_file)
print(f"Assignments saved to {output_file}")


Assignments saved to updated_assignments.xlsx
